In [ ]:
// nuget references
#r "nuget: FSharp.Stats, 0.5.1-preview.1"
//#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.1"
#r "nuget: FSharp.Data, 6.3.0"
#r "nuget: Cytoscape.NET, 0.2.0"
#r "nuget: Cytoscape.NET.Interactive, 0.2.0"

open FSharp.Stats
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open FSharp.Data
open Cytoscape.NET
open System

// use a script to import data in all notebooks > 1
#load "import.fsx"
open Import

let orders = Import.orders



Some of FSharp.Stats functionalities require the usage of [LAPACK](https://www.netlib.org/lapack/) routines. After the initial package download you can find these at `C:\Users\USERNAME\.nuget\packages\fsharp.stats\0.5.1-preview.1\netlib_LAPACK`. In the prepared use cases it is not necessary to load it but if you want, the next two lines do the job 

In [ ]:
//FSharp.Stats.ServiceLocator.setEnvironmentPathVariable (@"C:\Users\USERNAME\.nuget\packages\fsharp.stats\0.5.1-preview.1\netlib_LAPACK")
//FSharp.Stats.Algebra.LinearAlgebra.Service()

## Social network generation

The data allows the construction of a social network of drinking partners. In theory drinking partners are likely to log their drinks within a short period of time. Of course this assumption is prone to error because there are two logging devices in different building, and additionally external factors (like the end of a lecture many people attend) are likely to cause simultaneous thirst.

To start this analysis we map over all orders, and for each order isolate orders that are within a short time period (e.g. 1 minute prior and after). From these orders we can isolate the user names because that the only thing we are currently interested in. To remove self references, an additional filter step is required. 


From there it's an easy task to determine the number of simultaneous drinking.

As discussed earlier, it is possible to become a drinking parter by chance. To reduce the probability of getting false positives, it is recommended to filter sparse relationships. 
Therefore you could either just set an arbitrary threshold (e.g. 3) or you could visualize the count distribution and make an educated guess about an appropriate threshold.

It becomes apparent, that most of the person-person relations have a simultaneous drink just a few times (<5). A threshold of e.g. 8 seems to be appropriate. Another thing you may noticed is the histogram counts are always multiple of 2.
This is due to the fact, that not only `(Hugo Green, Abigail Payne)` has a drinking count of `21`, but also `(Abigail Payne, Hugo Green)`.
Because we are not interested in a directed network where it does matter who took a drink first, we just can ignore half of the data: 

Create a chart that visualizes these counts.

We have additional information about the department the users are working in. This can be used to color the nodes respectively. Create function that takes a department name and returns a color string in the format "#fffff". Additionally create a Map that returns a deparment color when given a user name. 

Now we have all node, edge and styling information to generate a graph. 

Please check out the documentation: https://fslab.org/Cytoscape.NET/.

Start by creating a function that takes the `((string*string)*int) []` of the filtered partner counts and returns a sequence of Cytoscape.NET.Elements.Node. For every user we need a single node element.


Do the same with the edges!

Now isolate nodes and edges from the drinking parter data and create a CyGraph.

Do some styling using user names and deparment colors. The weights of the edge could represent the count of simultanous drinking.

## Correlation network

Besides a social network we can also generate a day-based correlation network. Here we assign high correlation scores to a user-user pair, if their drinking behaviour is similar.

While this seems like that this analysis will show the same results as the network before, the readout will be different.

The most common correlation measure is the Pearsons correlation coefficient. It ranges from -1 to 1, while 0 indicates no correlation at all and 1 indicates a perfect correlation of two collections.

To be able to calculate correlations between two people, you could encode days when someone logged a drink by 1 and 0 otherwise.

Task: Create a nested collection as float [] [], that for each person contains an array of 1., -1 or 0..

```fsharp
[ //     | drank some kind of beverage on the second day since logging start
    [0.; 1.; -1.; ...] //Nicholas Powell
    [0.; 1.; 0.; ...] //Timo M.
    [-1.; 0.; 1.; ...] //Chloe Perkins
]
```


Use a density point chart to visualize the encoding pairs of e.g. Benedikt (index 4) and Kevin (index 15).
You can additionally calculate the pearson correlation coefficient and do proper styling.

The encoded matrix already is in a form that can be used to calculate a pairwise pearson correlation matrix.

Use an appropriate function from the FSharp.Stats.Correlation module and visualize the resulting correlation matrix as heatmap.

By calling `Matrix.ofJaggedArray` or just `matrix` for short, you can convert this jagged array into a matrix.

To get an intuition of what the correlation distribution looks like, you can create a histogram out of the correlation coefficients. Note, that you should filter values of 1. since they are self-references.

A method to identify an appropriate threshold for many types of correlation networks is called Random Matrix Theory. More indepth descriptions you can find [here](https://fslab.org/blog/posts/correlation-network.html).
The threshold for the coffee correlation network is precomputed because of runtime and LAPACK dependency.

Iterate over the correlation matrix, and whenever a correlation exceeds the threshold, create nodes for the people and an edge between them. 